# Leukemia Blood Cell Classification

This notebook explores different neural network architectures for classifying microscopic blood cell images as either healthy (HEM) or leukemia blasts (ALL).

## Experiments
1. VGG16 (Transfer Learning)
2. Custom CNN with 32 filters
3. Custom CNN with 16 filters  
4. CNN with Dropout
5. EfficientNetB3 (Transfer Learning)

In [0]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

# Add parent directory to path for imports
import sys
sys.path.append('..')

from src.data_loader import load_data, get_sample_images
from src.models import get_model

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {len(tf.config.list_physical_devices('GPU')) > 0}")

In [0]:
# Configuration
DATA_DIR = '../data/C-NMC_Leukemia'
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 42

np.random.seed(SEED)
tf.random.set_seed(SEED)

## Data Loading & Exploration

In [0]:
# Load data
train_gen, val_gen, test_gen = load_data(
    DATA_DIR, 
    img_size=IMG_SIZE, 
    batch_size=BATCH_SIZE
)

print(f"Training samples: {train_gen.samples}")
print(f"Validation samples: {val_gen.samples}")
print(f"Test samples: {test_gen.samples}")
print(f"\nClass mapping: {train_gen.class_indices}")

In [0]:
# Visualize sample images
samples = get_sample_images(DATA_DIR, n_samples=4)

fig, axes = plt.subplots(2, 4, figsize=(12, 6))
fig.suptitle('Sample Blood Cell Images', fontsize=14)

for i, (label, paths) in enumerate(samples.items()):
    for j, path in enumerate(paths[:4]):
        img = plt.imread(path)
        axes[i, j].imshow(img)
        axes[i, j].axis('off')
        if j == 0:
            axes[i, j].set_title(f"{'Leukemia (ALL)' if label == 'all' else 'Normal (HEM)'}")

plt.tight_layout()
plt.show()

## Helper Functions

In [0]:
def plot_training_history(history, title=''):
    """Plot training and validation accuracy/loss curves."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    # Accuracy
    ax1.plot(history.history['accuracy'], label='Train')
    ax1.plot(history.history['val_accuracy'], label='Validation')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.set_title(f'{title} - Accuracy')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Loss
    ax2.plot(history.history['loss'], label='Train')
    ax2.plot(history.history['val_loss'], label='Validation')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.set_title(f'{title} - Loss')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


def evaluate_model(model, test_gen, model_name='Model'):
    """Evaluate model and print results."""
    loss, accuracy = model.evaluate(test_gen, verbose=0)
    print(f"\n{model_name} Results:")
    print(f"  Test Accuracy: {accuracy:.4f}")
    print(f"  Test Loss: {loss:.4f}")
    return accuracy, loss

---
## Model 1: VGG16 (Transfer Learning)

Using pretrained VGG16 with frozen weights. This model failed to learn, converging at 50% accuracy (random guessing).

In [0]:
# Build VGG16 model
model_vgg = get_model('vgg16')
model_vgg.summary()

In [0]:
# Train VGG16
history_vgg = model_vgg.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5,
    verbose=1
)

In [0]:
plot_training_history(history_vgg, 'VGG16')
evaluate_model(model_vgg, test_gen, 'VGG16')

**Analysis:** VGG16 converged at exactly 50% accuracy—essentially random guessing. The pretrained ImageNet weights don't transfer well to microscopic blood cell images without fine-tuning.

---
## Model 2: Custom CNN (32 Filters)

4-layer CNN with 32 filters, alternating kernel sizes of 3 and 1.

In [0]:
# Build CNN with 32 filters
model_cnn32 = get_model('cnn_32')
model_cnn32.summary()

In [0]:
# Train CNN-32
history_cnn32 = model_cnn32.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    verbose=1
)

In [0]:
plot_training_history(history_cnn32, 'CNN (32 Filters)')
evaluate_model(model_cnn32, test_gen, 'CNN-32')

**Analysis:** Achieved ~79% accuracy but showed diminishing returns after epoch 10. The model reached ~76% by epoch 1, suggesting most learning happened early. 5-10 epochs would have been more efficient.

---
## Model 3: Custom CNN (16 Filters)

Same architecture as Model 2 but with smaller filters—testing the accuracy/speed tradeoff.

In [0]:
# Build CNN with 16 filters
model_cnn16 = get_model('cnn_16')
model_cnn16.summary()

In [0]:
# Train CNN-16
history_cnn16 = model_cnn16.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,  # Originally tried 30, but 20 was optimal
    verbose=1
)

In [0]:
plot_training_history(history_cnn16, 'CNN (16 Filters)')
evaluate_model(model_cnn16, test_gen, 'CNN-16')

**Analysis:** Slightly lower accuracy (~78%) but faster training. The learning curve suggested 20 epochs was optimal. Good option when compute is limited.

---
## Model 4: CNN with Dropout

Three convolutional layers (32→16→8) with dropout regularization.

In [0]:
# Build CNN with dropout
model_dropout = get_model('cnn_dropout')
model_dropout.summary()

In [0]:
# Train CNN with dropout
history_dropout = model_dropout.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15,
    verbose=1
)

In [0]:
plot_training_history(history_dropout, 'CNN with Dropout')
evaluate_model(model_dropout, test_gen, 'CNN-Dropout')

**Analysis:** Dropout actually hurt performance (~74% accuracy). This suggests the model was already struggling to fit the data—regularization made it harder to learn useful features.

---
## Model 5: EfficientNetB3 (Transfer Learning)

EfficientNet uses compound scaling to balance network width, depth, and resolution. Best performing model.

In [0]:
# Build EfficientNetB3 model
model_effnet = get_model('efficientnet')
model_effnet.summary()

In [0]:
# Train EfficientNetB3
history_effnet = model_effnet.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    verbose=1
)

In [0]:
plot_training_history(history_effnet, 'EfficientNetB3')
evaluate_model(model_effnet, test_gen, 'EfficientNetB3')

**Analysis:** Best accuracy at ~98%! However, the high loss suggests overfitting. For production use, would need more regularization or data augmentation.

---
## Results Summary

In [0]:
# Summary comparison
results = {
    'VGG16': {'accuracy': 0.50, 'notes': 'Failed to learn'},
    'CNN-32': {'accuracy': 0.79, 'notes': 'Overtrained, 5-10 epochs sufficient'},
    'CNN-16': {'accuracy': 0.78, 'notes': 'Faster, similar results'},
    'CNN-Dropout': {'accuracy': 0.74, 'notes': 'Regularization hurt performance'},
    'EfficientNetB3': {'accuracy': 0.98, 'notes': 'Best accuracy, high loss'},
}

print("\n" + "="*60)
print("MODEL COMPARISON")
print("="*60)
print(f"{'Model':<15} {'Accuracy':<12} {'Notes'}")
print("-"*60)
for model, data in results.items():
    print(f"{model:<15} {data['accuracy']:.1%}        {data['notes']}")

In [0]:
# Visualize results
models = list(results.keys())
accuracies = [r['accuracy'] for r in results.values()]

plt.figure(figsize=(10, 5))
bars = plt.bar(models, accuracies, color=['#e74c3c', '#3498db', '#3498db', '#9b59b6', '#2ecc71'])
plt.ylabel('Test Accuracy')
plt.title('Model Comparison')
plt.axhline(y=0.5, color='gray', linestyle='--', label='Random Baseline')
plt.ylim(0, 1.1)

for bar, acc in zip(bars, accuracies):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
             f'{acc:.1%}', ha='center', fontsize=11)

plt.legend()
plt.tight_layout()
plt.show()

## Key Takeaways

1. **Transfer learning success varies dramatically** — VGG16 completely failed while EfficientNetB3 achieved 98% accuracy on the same task.

2. **More epochs isn't always better** — Most models showed diminishing returns after 5-10 epochs. Monitoring validation loss is crucial.

3. **Regularization can hurt underfitting models** — Dropout reduced accuracy when the base model was already struggling.

4. **Accuracy vs. compute tradeoffs exist** — A simple CNN achieved 79% accuracy in minutes; EfficientNet got 98% but required more resources.

5. **High accuracy ≠ production ready** — The best model (EfficientNet) showed signs of overfitting that would need addressing.

In [0]:
# Save best model (optional)
# model_effnet.save('../models/efficientnet_best.h5')